In [1]:
import pandas as pd

#data = pd.read_json("all.list3.json")

In [2]:

# read the entire file into a python array
with open('all.list3.json', 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
#data_df = pd.read_json(data)

TypeError: sequence item 0: expected str instance, bytes found

In [ ]:
data

In [ ]:
data2 = pd.read_csv("lm_geneid_prid.all.nonredun.1to1.txt", header=None,delim_whitespace=True)

data2 = data2.rename(columns = {

    0:'geneid',
    1:'prid',
})

data2_dict = data2.set_index('prid')['geneid'].to_dict()

In [ ]:
HEPG2_10=data[(data.cell_id=="HEPG2") & (data.pert_idose=="10 ÂµM")]

pertuniq=HEPG2_10.pert_id.unique()

df_up = pd.DataFrame()
df_dn = pd.DataFrame()

for i in range(0, len(pertuniq)): 
    HEPG2_10_pert=HEPG2_10[HEPG2_10.pert_id==pertuniq[i]]

    if len(HEPG2_10.distil_cc_q75) > 1:
        HEPG2_10_pert_q75=HEPG2_10_pert.distil_cc_q75.max() 
        HEPG2_10_pert_q75=HEPG2_10_pert[HEPG2_10_pert.distil_cc_q75==HEPG2_10_pert_q75]
        if len(HEPG2_10_pert_q75.distil_ss) > 1:
            HEPG2_10_pert_ss=HEPG2_10_pert_q75.distil_ss.max() 
            HEPG2_10_pert_best=HEPG2_10_pert_q75[HEPG2_10_pert_q75.distil_ss==HEPG2_10_pert_ss]
        elif len(HEPG2_10_pert_q75.distil_ss) == 1:
            HEPG2_10_pert_best=HEPG2_10_pert_q75
        else:
            print('something wrong!')
    elif len(HEPG2_10_pert.distil_cc_q75) == 1:
        HEPG2_10_pert_best=HEPG2_10_pert
    else:
        print('something wrong!')

    ngenes_dn=HEPG2_10_pert_best.ngenes_modulated_dn_lm.values[0]
    ngenes_up=HEPG2_10_pert_best.ngenes_modulated_up_lm.values[0]-1

    up_lm50_n=HEPG2_10_pert_best.up50_lm.values[0][0:(ngenes_up)]
    dn_lm50_n=HEPG2_10_pert_best.dn50_lm.values[0][-(ngenes_dn):]
    up_lm50_n_s= pd.Series(up_lm50_n)
    dn_lm50_n_s= pd.Series(dn_lm50_n)
    
    up_lm50_n_geneid = up_lm50_n_s.map(data2_dict)
    dn_lm50_n_geneid = dn_lm50_n_s.map(data2_dict)
    
    up_genes = pd.DataFrame({'labels': pertuniq[i] , 'ngenes_up': up_lm50_n_geneid})
    dn_genes = pd.DataFrame({'labels': pertuniq[i] , 'ngenes_dn': dn_lm50_n_geneid})
    df_up=df_up.append(up_genes)
    df_dn=df_dn.append(dn_genes)

df_up.to_csv("HEPG2_10_results_up.txt", sep=" ", header=False, index=False)
df_dn.to_csv("HEPG2_10_results_dn.txt", sep=" ", header=False, index=False)

In [ ]:
data.info()

In [ ]:
test = pd.read_json("all.list.1.json")
test.info()

In [ ]:
pertuniq=test.pert_id.unique()
len(pertuniq)